## Health Insurance Fraud

![](schema_draft_3.png)

In [1]:
from py2neo import Graph, Node
db = Graph(scheme="bolt", host="localhost", port=7687, secure=False, 
                auth=('neo4j', 'test'))
#db.delete_all()

In [2]:
import pandas as pd


In [3]:
%ls data

 Le volume dans le lecteur C nâ€™a pas de nom.
 Le numÃ©ro de sÃ©rie du volume est EE44-8BAE

 RÃ©pertoire de C:\Users\LENOVO\Documents\GitHub\healthcare_fraud\data

14/12/2021  10:13    <DIR>          .
14/12/2021  10:13    <DIR>          ..
23/10/2019  08:12         5Â 325Â 982 Test_Beneficiarydata-1542969243754.csv
23/10/2019  08:12         2Â 020Â 712 Test_Inpatientdata-1542969243754.csv
23/10/2019  08:12        18Â 806Â 934 Test_Outpatientdata-1542969243754.csv
23/10/2019  08:12            14Â 894 Test-1542969243754.csv
23/10/2019  08:12        11Â 444Â 821 Train_Beneficiarydata-1542865627584.csv
23/10/2019  08:12         8Â 572Â 555 Train_Inpatientdata-1542865627584.csv
23/10/2019  08:12        77Â 391Â 090 Train_Outpatientdata-1542865627584.csv
23/10/2019  08:12            87Â 094 Train-1542865627584.csv
               8 fichier(s)      123Â 664Â 082 octets
               2 RÃ©p(s)  380Â 506Â 345Â 472 octets libres


In [4]:
df=pd.read_csv("data/Train_Beneficiarydata-1542865627584.csv")

In [5]:
df.columns

Index(['BeneID', 'DOB', 'DOD', 'Gender', 'Race', 'RenalDiseaseIndicator',
       'State', 'County', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt'],
      dtype='object')

In [6]:
df.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,...,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NaN,2,1,0,39,280,12,12,...,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NaN,1,1,0,52,590,12,12,...,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NaN,1,1,0,39,270,12,12,...,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NaN,1,1,0,24,680,12,12,...,2,1,2,2,2,2,0,0,1790,1200


## Beneficiary nodes


In [7]:
beneficiaries = list(df.BeneID.unique())
beneficiaries[:3]

['BENE11001', 'BENE11002', 'BENE11003']

In [8]:
len(beneficiaries)

138556

In [9]:
bens = [{'idx': x} for x in beneficiaries]

In [38]:
bens[:3]

[{'idx': 'BENE11001'}, {'idx': 'BENE11002'}, {'idx': 'BENE11003'}]

In [39]:
# set constraint on beneficiary node that idx is unique
query="""CREATE CONSTRAINT ON (n:Beneficiary) ASSERT n.idx IS UNIQUE"""

In [ ]:
db.run(query)

In [41]:
query = """
    UNWIND $bens AS ben
    MERGE (b:Beneficiary {idx: ben.idx})
        """
db.run(query, {"bens":bens})

(No data)

## Provider nodes

In [10]:
df_in = pd.read_csv("data/Train_Inpatientdata-1542865627584.csv")

In [11]:
df_in.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6'],
      dtype='object')

In [12]:
df_out = pd.read_csv("data/Train_Outpatientdata-1542865627584.csv")

In [13]:
df_out.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode'],
      dtype='object')

In [18]:
providers = list(set(list(df_in.Provider.unique()) + list(df_out.Provider.unique())))

In [19]:
len(providers)

5410

In [20]:
providers[:4]

['PRV57088', 'PRV57251', 'PRV56718', 'PRV53015']

In [21]:
provs=[{"idx": x} for x in providers]

In [22]:
# set constraint on beneficiary node that idx is unique
query="""CREATE CONSTRAINT ON (n:Provider) ASSERT n.idx IS UNIQUE"""

In [23]:
db.run(query)

(No data)

In [24]:
query = """
    UNWIND $provs AS prov
    MERGE (b:Provider {idx: prov.idx})
        """
db.run(query, {"provs":provs})

(No data)

## (Provider)-[:HAS_BENEFICIARY]->(Beneficiary)

In [14]:
df_in["type"]="Inpatient"

In [15]:
df_out['type']="Outpatient"

In [16]:
df_all=df_in.append(df_out)

In [28]:
len(df_all)

558211

In [29]:
df_all.type.value_counts()

Outpatient    517737
Inpatient      40474
Name: type, dtype: int64

In [30]:
len(df_in)

40474

In [31]:
len(df_out)

517737

In [32]:
grp=df_all.groupby(['Provider','BeneID'])['ClaimID'].count()

In [33]:
grp=grp.reset_index()


In [34]:
grp.head()

,Provider,BeneID,ClaimID
0,PRV51001,BENE102690,1
1,PRV51001,BENE106078,1
2,PRV51001,BENE11727,1
3,PRV51001,BENE123416,1
4,PRV51001,BENE126477,1


In [35]:
grp['ClaimID'].value_counts()

1     274312
2      46849
3      18330
4       9347
5       5303
6       3255
7       2105
8       1350
9        888
10       566
11       379
12       222
13       143
14        92
15        61
16        41
17        26
18        15
19         8
20         3
21         2
23         2
28         1
Name: ClaimID, dtype: int64

In [36]:
rels = [{'origin':x[1]['Provider'],
       'target': x[1]['BeneID']} for x in grp.iterrows()]

In [37]:
len(rels)

363300

In [38]:
query = """
    UNWIND $rels AS rel
    MATCH (e1:Provider {idx: rel.origin})
    MATCH (e2:Beneficiary {idx: rel.target}) 
    MERGE (e1)-[m:HAS_BENEFICIARY]->(e2)"""

db.run(query, {"rels":rels})

(No data)

## Race nodes

In [39]:
races= list(df.Race.unique())

In [40]:
races

[1, 2, 3, 5]

In [41]:
rs = [{"race": int(x),
     "idx": i}  for (i,x) in enumerate(races)]

In [42]:
rs

[{'race': 1, 'idx': 0},
 {'race': 2, 'idx': 1},
 {'race': 3, 'idx': 2},
 {'race': 5, 'idx': 3}]

In [43]:
query=""" CREATE CONSTRAINT ON (r:Race) ASSERT r.idx is UNIQUE """

In [44]:
db.run(query)

(No data)

In [45]:
query = """
    UNWIND $rs AS r
    MERGE (b:Race {idx: r.idx, race: r.race})
        """
db.run(query, {"rs":rs})

(No data)

In [46]:
query = """
    MATCH (r:Race)
    RETURN * """
db.run(query).to_table()

r
"(_143966:Race {idx: 0, race: 1})"
"(_143967:Race {idx: 1, race: 2})"
"(_143968:Race {idx: 2, race: 3})"
"(_143969:Race {idx: 3, race: 5})"


## (Beneficiary)-[:HAS_RACE]->(Race)

In [47]:
grp=df[['BeneID','Race']]

In [48]:
grp

,BeneID,Race
0,BENE11001,1
1,BENE11002,1
2,BENE11003,1
3,BENE11004,1
4,BENE11005,1
...,...,...
138551,BENE159194,1
138552,BENE159195,1
138553,BENE159196,1
138554,BENE159197,1


In [49]:
rels = [{'origin': x[1]['BeneID'],
       'target':  x[1]['Race']} for x in grp.iterrows()]

In [50]:
query = """
    UNWIND $rels AS rel
    MATCH (e1:Beneficiary {idx: rel.origin})
    MATCH (e2:Race {idx: rel.target}) 
    MERGE (e1)-[m:HAS_RACE]->(e2)"""

db.run(query, {"rels":rels})

(No data)

## Gender

In [51]:
genders=list(df.Gender.unique())

In [52]:
genders

[1, 2]

In [53]:
gs = [{"gender": int(x),
     "idx" : i} for (i,x) in enumerate(genders)]

In [54]:
gs

[{'gender': 1, 'idx': 0}, {'gender': 2, 'idx': 1}]

In [55]:
query="""CREATE CONSTRAINT ON (n:Gender) ASSERT n.idx IS UNIQUE"""

In [56]:
db.run(query)

(No data)

In [57]:
query = """
    UNWIND $gs AS g
    MERGE (b:Gender {idx: g.idx, gender: g.gender})
        """
db.run(query, {"gs":gs})

(No data)

## (Beneficiary)-[:HAS_GENDER]->(Gender)

In [58]:
grp=df[['BeneID','Gender']]

In [59]:
grp

,BeneID,Gender
0,BENE11001,1
1,BENE11002,2
2,BENE11003,1
3,BENE11004,1
4,BENE11005,1
...,...,...
138551,BENE159194,1
138552,BENE159195,2
138553,BENE159196,2
138554,BENE159197,1


In [60]:
rels = [{'origin': x[1]['BeneID'],
       'target':  x[1]['Gender']} for x in grp.iterrows()]

In [61]:
query = """
    UNWIND $rels AS rel
    MATCH (e1:Beneficiary {idx: rel.origin})
    MATCH (e2:Gender {idx: rel.target}) 
    MERGE (e1)-[m:HAS_GENDER]->(e2)"""

db.run(query, {"rels":rels})

(No data)

## State

In [62]:
states=list(df.State.unique())

In [63]:
sts = [{"state": int(x),
     "idx" : i} for (i,x) in enumerate(states)]

In [64]:
query="""CREATE CONSTRAINT ON (n:State) ASSERT n.idx IS UNIQUE"""

In [65]:
db.run(query)

(No data)

In [66]:
query = """
    UNWIND $sts AS st
    MERGE (b:State {idx: st.idx, state: st.state})
        """
db.run(query, {"sts":sts})

(No data)

## (Beneficiary)-[:HOME_STATE]->(STATE)

In [67]:
grp=df[['BeneID','State']]

In [68]:
grp

,BeneID,State
0,BENE11001,39
1,BENE11002,39
2,BENE11003,52
3,BENE11004,39
4,BENE11005,24
...,...,...
138551,BENE159194,39
138552,BENE159195,49
138553,BENE159196,6
138554,BENE159197,16


In [69]:
rels = [{'origin': x[1]['BeneID'],
       'target':  x[1]['State']} for x in grp.iterrows()]

In [70]:
query = """
    UNWIND $rels AS rel
    MATCH (e1:Beneficiary {idx: rel.origin})
    MATCH (e2:State {idx: rel.target}) 
    MERGE (e1)-[m:HOME_STATE]->(e2)"""

db.run(query, {"rels":rels})

(No data)

## County

In [71]:
counties=list(df.County.unique())

In [72]:
cts = [{"county": int(x),
     "idx" : i} for (i,x) in enumerate(counties)]

In [73]:
query="""CREATE CONSTRAINT ON (n:County) ASSERT n.idx IS UNIQUE"""

In [74]:
db.run(query)

(No data)

In [75]:
query = """
    UNWIND $cts AS ct
    MERGE (b:County {idx: ct.idx, county: ct.county})
        """
db.run(query, {"cts":cts})

(No data)

## (Beneficiary)-[:HOME_COUNTY]->(County)

In [76]:
grp=df[['BeneID','County']]

In [77]:
grp

,BeneID,County
0,BENE11001,230
1,BENE11002,280
2,BENE11003,590
3,BENE11004,270
4,BENE11005,680
...,...,...
138551,BENE159194,140
138552,BENE159195,530
138553,BENE159196,150
138554,BENE159197,560


In [78]:
rels = [{'origin': x[1]['BeneID'],
       'target':  x[1]['County']} for x in grp.iterrows()]

In [79]:
query = """
    UNWIND $rels AS rel
    MATCH (e1:Beneficiary {idx: rel.origin})
    MATCH (e2:County {idx: rel.target}) 
    MERGE (e1)-[m:HOME_COUNTY]->(e2)"""

db.run(query, {"rels":rels})

(No data)

## Months_PartA

In [80]:
Amonths=list(df.NoOfMonths_PartACov.unique())

In [81]:
ams = [{"Months_PartA": int(x),
     "idx" : i} for (i,x) in enumerate(Amonths)]

In [82]:
query="""CREATE CONSTRAINT ON (n:Months_PartA) ASSERT n.idx IS UNIQUE"""

In [83]:
db.run(query)

(No data)

In [84]:
query = """
    UNWIND $ams AS am
    MERGE (b:Months_PartA {idx: am.idx, Months_PartA: am.Months_PartA})
        """
db.run(query, {"ams":ams})

(No data)

## (Beneficiary)-[:HAS_COVERAGE_A]->(Months_PartA)

In [85]:
grp=df[['BeneID','NoOfMonths_PartACov']]

In [86]:
grp

,BeneID,NoOfMonths_PartACov
0,BENE11001,12
1,BENE11002,12
2,BENE11003,12
3,BENE11004,12
4,BENE11005,12
...,...,...
138551,BENE159194,12
138552,BENE159195,12
138553,BENE159196,12
138554,BENE159197,12


In [87]:
rels = [{'origin': x[1]['BeneID'],
       'target':  x[1]['NoOfMonths_PartACov']} for x in grp.iterrows()]

In [88]:
query = """
    UNWIND $rels AS rel
    MATCH (e1:Beneficiary {idx: rel.origin})
    MATCH (e2:Months_PartA {idx: rel.target}) 
    MERGE (e1)-[m:HAS_COVERAGE_A]->(e2)"""

db.run(query, {"rels":rels})

(No data)

## Months_PartB

In [89]:
Bmonths=list(df.NoOfMonths_PartBCov.unique())

In [90]:
bms = [{"Months_PartB": int(x),
     "idx" : i} for (i,x) in enumerate(Bmonths)]

In [91]:
query="""CREATE CONSTRAINT ON (n:Months_PartB) ASSERT n.idx IS UNIQUE"""

In [92]:
db.run(query)

(No data)

In [93]:
query = """
    UNWIND $bms AS bm
    MERGE (b:Months_PartB {idx: bm.idx, Months_PartB: bm.Months_PartB})
        """
db.run(query, {"bms":bms})

(No data)

## (Beneficiary)-[:HAS_COVERAGE_B]->(Months_PartB)

In [94]:
grp=df[['BeneID','NoOfMonths_PartBCov']]

In [95]:
rels = [{'origin': x[1]['BeneID'],
       'target':  x[1]['NoOfMonths_PartBCov']} for x in grp.iterrows()]

In [96]:
query = """
    UNWIND $rels AS rel
    MATCH (e1:Beneficiary {idx: rel.origin})
    MATCH (e2:Months_PartB {idx: rel.target}) 
    MERGE (e1)-[m:HAS_COVERAGE_B]->(e2)"""

db.run(query, {"rels":rels})

(No data)

## RenalDiseaseIndicator

In [97]:
renals=list(df.RenalDiseaseIndicator.unique())

In [98]:
renals

['0', 'Y']

In [99]:
rns = [{"Renal_Disease": x,
     "idx" : i} for (i,x) in enumerate(renals)]

In [100]:
query="""CREATE CONSTRAINT ON (n:Renal_Disease) ASSERT n.idx IS UNIQUE"""

In [101]:
db.run(query)

(No data)

In [102]:
query = """
    UNWIND $rns AS rn
    MERGE (b:Renal_Disease {idx: rn.idx, Renal_Disease: rn.Renal_Disease})
        """
db.run(query, {"rns":rns})

(No data)

## (Beneficiary)-[:HAS_CHRONIC_R]->(Renal_Disease)

In [103]:
grp=df[['BeneID','RenalDiseaseIndicator']]

In [104]:
rels = [{'origin': x[1]['BeneID'],
       'target':  x[1]['RenalDiseaseIndicator']} for x in grp.iterrows()]

In [105]:
query = """
    UNWIND $rels AS rel
    MATCH (e1:Beneficiary {idx: rel.origin})
    MATCH (e2:Renal_Disease {idx: rel.target}) 
    MERGE (e1)-[m:HAS_CHRONIC_R]->(e2)"""

db.run(query, {"rels":rels})

(No data)

## ChronicCond_Alzheimer

In [106]:
alzheimers=list(df.ChronicCond_Alzheimer.unique())

In [107]:
alzs = [{"ChronicCond_Alzheimer": int(x),
     "idx" : i} for (i,x) in enumerate(alzheimers)]

In [108]:
query="""CREATE CONSTRAINT ON (n:ChronicCond_Alzheimer) ASSERT n.idx IS UNIQUE"""

In [109]:
db.run(query)

(No data)

In [110]:
query = """
    UNWIND $alzs AS alz
    MERGE (b:ChronicCond_Alzheimer {idx: alz.idx, ChronicCond_Alzheimer: alz.ChronicCond_Alzheimer})
        """
db.run(query, {"alzs":alzs})

(No data)

## (Beneficiary)-[:HAS_CHRONIC_A]->(ChronicCond_Alzheimer)

In [111]:
grp=df[['BeneID','ChronicCond_Alzheimer']]

In [112]:
grp

,BeneID,ChronicCond_Alzheimer
0,BENE11001,1
1,BENE11002,2
2,BENE11003,1
3,BENE11004,1
4,BENE11005,2
...,...,...
138551,BENE159194,1
138552,BENE159195,1
138553,BENE159196,2
138554,BENE159197,1


In [113]:
rels = [{'origin': x[1]['BeneID'],
       'target':  x[1]['ChronicCond_Alzheimer']} for x in grp.iterrows()]

In [114]:
query = """
    UNWIND $rels AS rel
    MATCH (e1:Beneficiary {idx: rel.origin})
    MATCH (e2:ChronicCond_Alzheimer {idx: rel.target}) 
    MERGE (e1)-[m:HAS_CHRONIC_A]->(e2)"""

db.run(query, {"rels":rels})

(No data)

## ChronicCond_Heartfailure

In [115]:
hearts=list(df.ChronicCond_Heartfailure.unique())

In [116]:
hs = [{"ChronicCond_Heartfailure": int(x),
     "idx" : i} for (i,x) in enumerate(hearts)]

In [117]:
query="""CREATE CONSTRAINT ON (n:ChronicCond_Heartfailure) ASSERT n.idx IS UNIQUE"""

In [118]:
db.run(query)

(No data)

In [119]:
query = """
    UNWIND $hs AS h
    MERGE (b:ChronicCond_Heartfailure {idx: h.idx, ChronicCond_Heartfailure: h.ChronicCond_Heartfailure})
        """
db.run(query, {"hs":hs})

(No data)

## (Beneficiary)-[:HAS_CHRONIC_H]->(ChronicCond_Heartfailure)

In [120]:
grp=df[['BeneID','ChronicCond_Heartfailure']]

In [121]:
rels = [{'origin': x[1]['BeneID'],
       'target':  x[1]['ChronicCond_Heartfailure']} for x in grp.iterrows()]

In [122]:
query = """
    UNWIND $rels AS rel
    MATCH (e1:Beneficiary {idx: rel.origin})
    MATCH (e2:ChronicCond_Heartfailure {idx: rel.target}) 
    MERGE (e1)-[m:HAS_CHRONIC_H]->(e2)"""

db.run(query, {"rels":rels})

(No data)

## ChronicCond_KidneyDisease

In [123]:
kidneys=list(df.ChronicCond_KidneyDisease.unique())

In [124]:
ks = [{"ChronicCond_KidneyDisease": int(x),
     "idx" : i} for (i,x) in enumerate(kidneys)]

In [125]:
query="""CREATE CONSTRAINT ON (n:ChronicCond_KidneyDisease) ASSERT n.idx IS UNIQUE"""

In [126]:
db.run(query)

(No data)

In [127]:
query = """
    UNWIND $ks AS k
    MERGE (b:ChronicCond_KidneyDisease {idx: k.idx, ChronicCond_KidneyDisease: k.ChronicCond_KidneyDisease})
        """
db.run(query, {"ks":ks})

(No data)

## (Beneficiary)-[:HAS_CHRONIC_K]->(ChronicCond_KidneyDisease)

In [128]:
grp=df[['BeneID','ChronicCond_KidneyDisease']]

In [129]:
rels = [{'origin': x[1]['BeneID'],
       'target':  x[1]['ChronicCond_KidneyDisease']} for x in grp.iterrows()]

In [130]:
query = """
    UNWIND $rels AS rel
    MATCH (e1:Beneficiary {idx: rel.origin})
    MATCH (e2:ChronicCond_KidneyDisease {idx: rel.target}) 
    MERGE (e1)-[m:HAS_CHRONIC_K]->(e2)"""

db.run(query, {"rels":rels})

(No data)

## IsFraudulent

In [55]:
providers=pd.read_csv("data/Train-1542865627584.csv")

In [56]:
providers

,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No
...,...,...
5405,PRV57759,No
5406,PRV57760,No
5407,PRV57761,No
5408,PRV57762,No


In [69]:
fraudulents=list(providers.PotentialFraud.unique())

In [70]:
fraudulents

['No', 'Yes']

In [71]:
fs = [{"IsFraudulent": x,
     "idx" : i} for (i,x) in enumerate(fraudulents)]

In [136]:
query="""CREATE CONSTRAINT ON (n:IsFraudulent) ASSERT n.idx IS UNIQUE"""

In [137]:
db.run(query)

(No data)

In [138]:
query = """
    UNWIND $fs AS f
    MERGE (b:IsFraudulent {idx: f.idx, IsFraudulent: f.IsFraudulent})
        """
db.run(query, {"fs":fs})

(No data)

## (Provider)-[:IS_FRAUDULENT]->(IsFraudulent)

In [57]:
providers

,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No
...,...,...
5405,PRV57759,No
5406,PRV57760,No
5407,PRV57761,No
5408,PRV57762,No


In [58]:
rels = [{'origin': x[1]['Provider'],
       'target':  x[1]['PotentialFraud']} for x in providers.iterrows()]

In [68]:
rels

[{'origin': 'PRV51001', 'target': 'No'},
 {'origin': 'PRV51003', 'target': 'Yes'},
 {'origin': 'PRV51004', 'target': 'No'},
 {'origin': 'PRV51005', 'target': 'Yes'},
 {'origin': 'PRV51007', 'target': 'No'},
 {'origin': 'PRV51008', 'target': 'No'},
 {'origin': 'PRV51011', 'target': 'No'},
 {'origin': 'PRV51012', 'target': 'No'},
 {'origin': 'PRV51013', 'target': 'No'},
 {'origin': 'PRV51014', 'target': 'No'},
 {'origin': 'PRV51015', 'target': 'No'},
 {'origin': 'PRV51016', 'target': 'No'},
 {'origin': 'PRV51017', 'target': 'No'},
 {'origin': 'PRV51021', 'target': 'Yes'},
 {'origin': 'PRV51023', 'target': 'No'},
 {'origin': 'PRV51024', 'target': 'No'},
 {'origin': 'PRV51025', 'target': 'No'},
 {'origin': 'PRV51026', 'target': 'No'},
 {'origin': 'PRV51027', 'target': 'No'},
 {'origin': 'PRV51029', 'target': 'No'},
 {'origin': 'PRV51030', 'target': 'No'},
 {'origin': 'PRV51031', 'target': 'No'},
 {'origin': 'PRV51032', 'target': 'No'},
 {'origin': 'PRV51035', 'target': 'No'},
 {'origin': '

In [74]:
query = """
    UNWIND $rels as rel
    MATCH (e1:Provider {idx: rel.origin})
    MATCH (e2:IsFraudulent {IsFraudulent: rel.target})
    MERGE (e1)-[m:IS_FRAUD]->(e2)

"""
db.run(query, {'rels': rels}).to_table()

## Claim nodes

In [28]:
df_all

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,type
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inpatient
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN,Inpatient
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inpatient
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN,Inpatient
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN,Inpatient
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517732,BENE159198,CLM510792,2009-08-06,2009-08-06,PRV53699,800,PHY364188,PHY364188,PHY385752,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outpatient
517733,BENE159198,CLM551294,2009-08-29,2009-08-29,PRV53702,400,PHY423019,PHY332284,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outpatient
517734,BENE159198,CLM596444,2009-09-24,2009-09-24,PRV53676,60,PHY361063,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outpatient
517735,BENE159198,CLM636992,2009-10-18,2009-10-18,PRV53689,70,PHY403198,NaN,PHY419379,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outpatient


In [53]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558211 entries, 0 to 517736
Data columns (total 31 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   BeneID                  558211 non-null  object 
 1   ClaimID                 558211 non-null  object 
 2   ClaimStartDt            558211 non-null  object 
 3   ClaimEndDt              558211 non-null  object 
 4   Provider                558211 non-null  object 
 5   InscClaimAmtReimbursed  558211 non-null  int64  
 6   AttendingPhysician      556703 non-null  object 
 7   OperatingPhysician      114447 non-null  object 
 8   OtherPhysician          199736 non-null  object 
 9   AdmissionDt             40474 non-null   object 
 10  ClmAdmitDiagnosisCode   145899 non-null  object 
 11  DeductibleAmtPaid       557312 non-null  float64
 12  DischargeDt             40474 non-null   object 
 13  DiagnosisGroupCode      40474 non-null   object 
 14  ClmDiagnosisCode_1  

In [15]:
claims = list(df_all.ClaimID.unique())
claims[:3]

['CLM46614', 'CLM66048', 'CLM68358']

In [16]:
len(claims)

558211

In [17]:
clms = [{'idx': x} for x in claims]

In [18]:
clms[:3]

[{'idx': 'CLM46614'}, {'idx': 'CLM66048'}, {'idx': 'CLM68358'}]

In [19]:
# set constraint on claim node that idx is unique
query="""CREATE CONSTRAINT ON (n:Claim) ASSERT n.idx IS UNIQUE"""

In [155]:
db.run(query)

(No data)

In [20]:
query = """CALL apoc.periodic.iterate(
    'UNWIND $clms AS clm MERGE (b:Claim {idx: clm.idx}) RETURN b',
    'WITH b RETURN count(b)',
    {batchSize:100, parallel:false, params: {clms:$clms}})"""

In [24]:
i=0
while i < len(claims):
    db.run(query, {"clms":clms[i:i+5000]}).to_table()
    i=i+5000

## Physician nodes:

In [34]:
phy_df = df_all[df_all['AttendingPhysician'].notna()]

In [36]:
physicians = list(phy_df.AttendingPhysician.unique())
physicians[:3]

['PHY390922', 'PHY318495', 'PHY372395']

In [37]:
len(physicians)

82063

In [38]:
phs = [{'idx': x} for x in physicians]

In [39]:
# set constraint on beneficiary node that idx is unique
query="""CREATE CONSTRAINT ON (n:Physician) ASSERT n.idx IS UNIQUE"""

In [146]:
db.run(query)

(No data)

In [40]:
query = """
    UNWIND $phs AS ph
    MERGE (b:Physician {idx: ph.idx})
        """
db.run(query, {"phs":phs})

(No data)

## (Provider)-[:HAS_WORKED_WITH]->(Physician)

In [41]:
grp=df_all.groupby(['Provider','AttendingPhysician'])['ClaimID'].count()

In [42]:
grp=grp.reset_index()

In [43]:
grp

,Provider,AttendingPhysician,ClaimID
0,PRV51001,PHY311126,1
1,PRV51001,PHY318842,1
2,PRV51001,PHY340163,1
3,PRV51001,PHY345302,4
4,PRV51001,PHY346762,1
...,...,...,...
87900,PRV57760,PHY398330,4
87901,PRV57761,PHY354815,68
87902,PRV57761,PHY367727,14
87903,PRV57762,PHY432730,1


In [44]:
rels = [{'origin':x[1]['Provider'],
       'target': x[1]['AttendingPhysician']} for x in grp.iterrows()]

In [45]:
query = """
    UNWIND $rels AS rel
    MATCH (e1:Provider {idx: rel.origin})
    MATCH (e2:Physician {idx: rel.target}) 
    MERGE (e1)-[m:HAS_WORKED_WITH]->(e2)"""

db.run(query, {"rels":rels})

(No data)

## Type Node

In [46]:
types=list(df_all.type.unique())

In [47]:
types

['Inpatient', 'Outpatient']

In [48]:
ts = [{"Type": x,
     "idx" : i} for (i,x) in enumerate(types)]

In [49]:
ts

[{'Type': 'Inpatient', 'idx': 0}, {'Type': 'Outpatient', 'idx': 1}]

In [50]:
query="""CREATE CONSTRAINT ON (n:Type) ASSERT n.idx IS UNIQUE"""

In [51]:
db.run(query)

(No data)

In [52]:
query = """
    UNWIND $ts AS t
    MERGE (b:Type {idx: t.idx, Type: t.Type})
        """
db.run(query, {"ts":ts})

(No data)

## Diagnosis Node: 